# Merge ERA5 1 deg model level and single level variables

In [46]:
import os
import sys
import yaml
import dask
import zarr
import numpy as np
import xarray as xr
import pandas as pd
from glob import glob

import calendar
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

sys.path.insert(0, os.path.realpath('../libs/'))
import verif_utils as vu

In [54]:
import matplotlib.pyplot as plt
%matplotlib inline

In [47]:
base_dir_mlevel = '/glade/derecho/scratch/ksha/CREDIT_data/ERA5_mlevel_1deg/'
base_dir_plevel = '/glade/derecho/scratch/ksha/CREDIT_data/ERA5_plevel_1deg/'
base_dir_output = base_dir_mlevel

In [48]:
fn_fmt_mlevel = base_dir_mlevel + 'upper_air/ERA5_mlevel_1deg_6h_{}_conserve.zarr'
fn_fmt_cloud = base_dir_mlevel + 'cloud/ERA5_mlevel_1deg_6h_cloud_{}_conserve.zarr'
fn_fmt_plevel = base_dir_plevel + 'all_in_one/ERA5_plevel_1deg_6h_{}_conserve.zarr'
fn_mean_std = '/glade/derecho/scratch/ksha/CREDIT_data/mean_6h_1979_2018_16lev_0.25deg.nc'

In [49]:
mlevel_picks = [1, 2, 4, 6, 9, 12, 15, 18, 21, 24, 27, 
                30, 33, 36, 39, 43, 47, 51, 54, 58, 62,
                66, 70, 74, 77, 81, 84, 87, 90, 93, 97, 
                100, 104, 107, 111, 114, 116, 119, 122, 
                124, 126, 128, 131, 133, 136, 137]

var_mlevel = {
    'specific_humidity': mlevel_picks,
    'temperature': mlevel_picks,
    'u_component_of_wind': mlevel_picks,
    'v_component_of_wind': mlevel_picks
}

var_cloud = {
    'specific_cloud_liquid_water_content': mlevel_picks,
    'specific_rain_water_content': mlevel_picks,
}

chunk_size_3d = {
    'time': 10,
    'latitude': 181,
    'longitude': 360
}

chunk_size_4d = {
    'time': 10,
    'level': 46,
    'latitude': 181,
    'longitude': 360
}

encode_size_3d = dict(
    chunks=(
        chunk_size_3d['time'],
        chunk_size_3d['latitude'],
        chunk_size_3d['longitude']
    )
)

encode_size_4d = dict(
    chunks=(
        chunk_size_4d['time'],
        chunk_size_4d['level'],
        chunk_size_4d['latitude'],
        chunk_size_4d['longitude']
    )
)

### Main routine

In [50]:
year = 2020

ds_mlevel = xr.open_zarr(fn_fmt_mlevel.format(year))
ds_mlevel_sub = vu.ds_subset_everything(ds_mlevel, var_mlevel)

ds_cloud = xr.open_zarr(fn_fmt_cloud.format(year))
ds_cloud_sub = vu.ds_subset_everything(ds_cloud, var_cloud)

Q = ds_mlevel_sub['specific_humidity'] + \
    ds_cloud_sub['specific_cloud_liquid_water_content'] + \
    ds_cloud_sub['specific_rain_water_content']

ds_mlevel_sub['specific_total_water'] = Q

ds_plevel = xr.open_zarr(fn_fmt_plevel.format(year))
ds_plevel_sub = ds_plevel.drop_vars(['U', 'V', 'T', 'Q', 'Z', 'specific_total_water'])

ds_plevel_sub = ds_plevel_sub.drop_vars(['level',])
ds_merge = xr.merge([ds_mlevel_sub, ds_plevel_sub])

varnames = list(ds_merge.keys())
varname_4D = [
    'specific_humidity',
    'temperature',
    'u_component_of_wind',
    'v_component_of_wind',
    'specific_total_water'
]

for i_var, var in enumerate(varnames):
    if var in varname_4D:
        ds_merge[var] = ds_merge[var].chunk(chunk_size_4d)
    else:
        ds_merge[var] = ds_merge[var].chunk(chunk_size_3d)

dict_encoding = {}

compress = zarr.Blosc(cname='zstd', clevel=1, shuffle=zarr.Blosc.SHUFFLE, blocksize=0)

for i_var, var in enumerate(varnames):
    if var in varname_4D:
        dict_encoding[var] = {'compressor': compress, **encode_size_4d}
    else:
        dict_encoding[var] = {'compressor': compress, **encode_size_3d}

save_name = base_dir_output + 'all_in_one/ERA5_mlevel_1deg_6h_subset_{}_conserve.zarr'.format(year)
# ds_merge.to_zarr(save_name, mode='w', consolidated=True, compute=True, encoding=dict_encoding)

### Static file

In [36]:
fn_fmt_static = base_dir_mlevel + 'static/ERA5_mlevel_1deg_6h_conserve_static.zarr'

In [37]:
chunk_size_2d = {
    'latitude': 181,
    'longitude': 360
}

chunk_size_1d = {'level': 46}

encode_size_2d = dict(
    chunks=(
        chunk_size_2d['latitude'],
        chunk_size_2d['longitude']
    )
)

encode_size_1d = dict(
    chunks=(
        chunk_size_4d['level'],
    )
)

In [38]:
ds_static = xr.open_zarr(fn_fmt_static)

ds_static['half_level'] = ds_static['half_level'].values - 1
ds_static = ds_static.sel(half_level=mlevel_picks)
ds_static = ds_static.rename({'half_level': 'level'})
ds_static = ds_static.astype({var: 'float32' for var in ds_static.data_vars})

varnames = list(ds_static.keys())
varname_1d = ['coef_a', 'coef_b']

dict_encoding = {}
    
compress = zarr.Blosc(cname='zstd', clevel=1, shuffle=zarr.Blosc.SHUFFLE, blocksize=0)
    
for i_var, var in enumerate(varnames):
    if var in varname_1d:
        dict_encoding[var] = {'compressor': compress, **encode_size_1d}
    else:
        dict_encoding[var] = {'compressor': compress, **encode_size_2d}

save_name = base_dir_output + 'static/ERA5_mlevel_1deg_static_subset.zarr'
# ds_static.to_zarr(save_name, mode='w', consolidated=True, compute=True, encoding=dict_encoding)